In [24]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import datetime
coviddata=pd.read_hdf('./covidtata.h5',key='updatefrom1130')

coviddata['new_cases_daily']=coviddata['新增无症状']+coviddata['新增本土确诊']
df_covid=coviddata[['省份', '城市', '日期','新增无症状', '新增本土确诊', '风险地区','new_cases_daily']]

s1=df_covid['日期']>"2022-02-01" #2022年2月1号开始区分 新增本土无症状（即表里的新增无症状）和新增本土确诊
s2=df_covid['省份'].isin(['北京','上海','天津','重庆'])
s3=~df_covid['城市'].isin(['北京','上海','天津','重庆'])

df_covid['城市']=df_covid['城市'].str.replace('伊犁州','伊犁')

df_covid=df_covid[s1] #去掉2022年前的
df_covid=df_covid[~(s2&s3)] #去掉直辖市分区的统计，因为后面没抓到其实，而且分析也没有意义

df_covid['日期'] = pd.to_datetime(df_covid['日期']).dt.date

def getyr(d):
    return str(d.isocalendar()[0])

# df_covid['yr']=df_covid['日期'].apply(getyr)

#rolling average
df_covid=df_covid.sort_values(by='日期',ascending=True).reset_index(drop=True)
df_covid['cases_last7d_max'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 1).max())
df_covid['cases_last3d_max'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(3, 1).max())



df_covid=df_covid.rename(columns={'日期':'date'})
df_covid=df_covid.sort_values(by='date',ascending=True).reset_index(drop=True)
df_covid['cases_last7d_sum'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 7).sum())
df_covid['cases_14_7d_sum'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(14, 14).sum()) - df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(7, 7).sum())

df_covid['cases_last3d_sum'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(3, 3).sum())
df_covid['cases_6_3d_sum'] = df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(6, 6).sum()) - df_covid.groupby('城市')['new_cases_daily'].transform(lambda x: x.rolling(3, 3).sum())


df_covid['case_growth_7']=df_covid['cases_last7d_sum']/df_covid['cases_14_7d_sum'] -1
df_covid['case_growth_3']=df_covid['cases_last3d_sum']/df_covid['cases_6_3d_sum'] -1


df_covid.tail(5)


,省份,城市,date,新增无症状,新增本土确诊,风险地区,new_cases_daily,cases_last7d_max,cases_last3d_max,cases_last7d_sum,cases_14_7d_sum,cases_last3d_sum,cases_6_3d_sum,case_growth_7,case_growth_3
106713,江西,萍乡,2022-12-07,0,0,0.0,0,3.0,1.0,6.0,3.0,1.0,5.0,1.000000,-0.800000
106714,江西,景德镇,2022-12-07,0,0,2.0,0,9.0,7.0,37.0,83.0,8.0,26.0,-0.554217,-0.692308
106715,北京,北京,2022-12-07,2194,1168,6919.0,3362,4666.0,4666.0,26229.0,26601.0,12002.0,10259.0,-0.013984,0.169900
106716,西藏,昌都,2022-12-07,0,0,11.0,0,0.0,0.0,0.0,8.0,0.0,0.0,-1.000000,NaN
106717,重庆,重庆,2022-12-07,3006,155,137.0,3161,6536.0,4168.0,33669.0,56031.0,11094.0,16039.0,-0.399100,-0.308311


In [25]:
# 1.1 疫情新增 by 人口 分析 

import numpy as np
city_popu=pd.read_hdf('./citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])
dfo=df_covid.merge(city_popu,on='城市',how='inner')

def catgorize(num):
    num=float(num)
    level=np.nan
    if num ==0:
        level= '0 无疫情区域'
    if 0 < num <=10:
        level ="1 轻微疫情区域"
    if num > 10 and num <=100:
        level ="2 较严重疫情区域"
    if num > 100:
        level ="3 严重疫情区域"

    return level

dfo['covid_level']=dfo['cases_last3d_max'].apply(catgorize)

show=dfo.copy()
c1=show['covid_level']==np.nan
show=show[~c1]

pv_value=pd.pivot_table(show, values='常住人口', index=['date'],columns=['covid_level'], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 无疫情区域', '1 轻微疫情区域', '2 较严重疫情区域', '3 严重疫情区域']]
pv_value.tail(22).to_clipboard()
pv_value.tail(5)

covid_level,sum,0 无疫情区域,1 轻微疫情区域,2 较严重疫情区域,3 严重疫情区域
date,,,,,
2022-12-03,1,0.031259,0.078103,0.463935,0.426703
2022-12-04,1,0.034502,0.073539,0.461367,0.430592
2022-12-05,1,0.036468,0.071792,0.463876,0.427863
2022-12-06,1,0.034972,0.073964,0.449113,0.441951
2022-12-07,1,0.040900,0.086482,0.466268,0.406349


In [26]:
# 2. Amap Traffic
# pd.read_excel('./Amap Conjestion.xlsx').to_hdf('./amap.h5',key='raw1129')

amap=pd.read_hdf('./amap.h5',key='raw1129')
amap=pd.melt(amap,id_vars=['城市','省','城市线'],var_name='date', value_name='traffic')
amap['date']=pd.to_datetime(amap['date']).dt.date


def getyr(d):
    return str(d.isocalendar()[0])
def getdt(d):
    return str(d)[-5:]

amap['yr']=amap['date'].apply(getyr)
amap['dt']=amap['date'].apply(getdt)

amap=amap.sort_values(by='date',ascending=True).reset_index(drop=True)

amap_2019=amap[amap['yr'].str.contains('2019')]
amap_2019['traffic_last30d_mean'] = amap_2019.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 1).mean()) # 当的traffic 变化

amap_2019=amap_2019[['省','城市','dt','traffic_last30d_mean']]
amap_2019=amap_2019.rename(columns={"traffic_last30d_mean":"traffic_19_rolling_mean"})

amap_2022=amap[amap['yr'].str.contains('2022')]
amap_2022=amap_2022.merge(amap_2019,how='left',on=['省','城市','dt'])

amap_2022['traffic_last7d_mean'] = amap_2022.groupby('城市')['traffic'].transform(lambda x: x.rolling(7, 7).mean())
amap_2022['traffic_status']=amap_2022['traffic']/amap_2022['traffic_19_rolling_mean']
amap_2022['traffic_change']=amap_2022['traffic_last7d_mean']/amap_2022.groupby('城市')['traffic_last7d_mean'].transform(lambda x: x.shift(periods=7))-1

amap_2022.tail(5)

,城市,省,城市线,date,traffic,yr,dt,traffic_19_rolling_mean,traffic_last7d_mean,traffic_status,traffic_change
33895,青岛,山东,二线,2022-12-07,1.82,2022,12-07,1.697143,1.675714,1.072391,0.001708
33896,苏州,江苏,二线,2022-12-07,1.58,2022,12-07,1.481429,1.508571,1.066538,-0.038251
33897,杭州,浙江,二线,2022-12-07,1.53,2022,12-07,1.501429,1.525714,1.019029,0.001876
33898,潍坊,山东,二线,2022-12-07,1.47,2022,12-07,1.567143,1.397143,0.938013,0.066521
33899,肇庆,广东,三线及以下,2022-12-07,1.49,2022,12-07,1.452857,1.405714,1.025565,-0.021869


In [29]:
# 2.1 Amap traffic change WoW vs. TY covid rank
dfo=amap_2022.merge(df_covid,on=['城市','date'],how='inner').merge(city_popu,on='城市',how='inner')
dfo['date']=dfo['date'].astype('string') 
dfo['covid_rank']=dfo.groupby(by=['date'])['cases_last7d_sum'].rank(method="dense", ascending=True)
o=dfo[dfo['date']=='2022-12-06'][['城市','covid_rank','traffic_change']] #,'case_growth_7','cases_last7d_sum','cases_14_7d_sum'


o['size']=np.nan
o['group']='B'
c1=o['城市'].isin(['广州','深圳','石家庄','天津','成都'])
o.loc[c1,'group']='A'

o.to_clipboard(index=False,header=None)

In [19]:
# 2.1 Amap traffic change /2019 vs. TY covid rank
dfo=amap_2022.merge(df_covid,on=['城市','date'],how='inner').merge(city_popu,on='城市',how='inner')
dfo['date']=dfo['date'].astype('string') 
dfo['covid_rank']=dfo.groupby(by=['date'])['cases_last7d_sum'].rank(method="dense", ascending=True)
dfo['traffic_status_lastXd'] = dfo.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(7, 1).mean())

o=dfo[dfo['date']=='2022-12-06'][['城市','covid_rank','traffic_status_lastXd']] #,'case_growth_7','cases_last7d_sum','cases_14_7d_sum'


o['size']=np.nan
o['group']='B'
c1=o['城市'].isin(['广州','深圳','石家庄','天津','成都'])
o.loc[c1,'group']='A'

o.to_clipboard(index=False,header=None)

In [23]:
# 地铁数据
pd.read_excel(r'./DataSource/地铁.xlsx').to_hdf('./subway.h5',key='raw')
subway=pd.read_hdf('./subway.h5',key='raw')
subway=pd.melt(subway,id_vars=['城市'],var_name='date',value_name='subway')

df_covid['date'] = pd.to_datetime(df_covid['date']).dt.date
subway['date'] = pd.to_datetime(subway['date']).dt.date
subway=subway.merge(df_covid,on=['城市','date'],how='left')
subway=subway.sort_values(by=['城市','date'],ascending=True)
subway['date']=subway['date'].astype('str')


def getdt(d):
    return str(d)[-5:]
subway['dt']=subway['date'].apply(getdt)

subway['subway_last7d_mean']=subway.groupby('城市')['subway'].transform(lambda x: x.rolling(7, 3).mean())
subway['subway_WoW']=subway['subway_last7d_mean']/subway.groupby('城市')['subway_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=subway['date'].str.contains('2019')
sw_19=subway[c1]
sw_19=sw_19[['城市', 'dt','subway_last7d_mean']]
sw_19=sw_19.rename(columns={'subway_last7d_mean':'subway_19_rolling_mean'})

c2=subway['date'].str.contains('2022')
sw_22=subway[c2]
sw_22=sw_22.merge(sw_19,how='left',on=['城市','dt'])

sw_22['subway_status']=sw_22['subway_last7d_mean']/sw_22['subway_19_rolling_mean']

sw_22['covid_rank']=sw_22.groupby(by=['date'])['cases_last7d_sum'].rank(method="dense", ascending=True)
o=sw_22[sw_22['subway_status']<2][sw_22['date']=='2022-12-06'][['城市','covid_rank','subway_status']] #,'case_growth_7','cases_last7d_sum','cases_14_7d_sum'

o['size']=np.nan
o['group']='B'
c1=o['城市'].isin(['广州','深圳','石家庄','天津','成都'])
o.loc[c1,'group']='A'

o.to_clipboard(index=False,header=None)

In [28]:
# covid rank vs. occ change
hotel=pd.read_hdf(r'./hotel.h5',key='raw1')
hotel['date']=pd.to_datetime(hotel['date']).dt.date
hotel['cityName']=hotel['cityName'].str.replace('市','')

hotel1=hotel.merge(city_popu,left_on='cityName',right_on='城市',how='inner').merge(df_covid,left_on=['cityName','date'],right_on=['城市','date'],how='inner')
hotel1['covid_rank']=hotel1.groupby(by=['date'])['cases_last7d_sum'].rank(method="dense", ascending=True)

hotel1=hotel1.sort_values(by=['cityName','date'])
hotel1['bookedNights_last7d_mean']=hotel1.groupby('cityName')['bookedNights'].transform(lambda x: x.rolling(7, 7).mean())
hotel1['bookedNights_change_WoW']=(hotel1['bookedNights_last7d_mean']/hotel1.groupby('cityName')['bookedNights_last7d_mean'].transform(lambda x: x.shift(periods=7)))-1

hotel1['date']=hotel1['date'].astype('string') 

hotel2=hotel1[hotel1['date']=='2022-12-06'].sort_values(by='bookedNights_last7d_mean',ascending=False)
hotel2=hotel2[hotel1['bookedNights_change_WoW']<1].head(100).reset_index()
# hote1
hotel2=hotel2[['cityName',  'covid_rank','bookedNights_change_WoW']]


hotel2['size']=np.nan
hotel2['group']='B'
c1=hotel2['cityName'].isin(['广州','深圳','石家庄','天津','成都'])
hotel2.loc[c1,'group']='A'
hotel2.to_clipboard(index=False,header=None)

In [97]:
city_popu=pd.read_hdf('./citypopu.h5',key='raw2')
city_popu['城市']=city_popu['城市'].str.replace('市',"")
city_popu['常住人口']=pd.to_numeric(city_popu['常住人口'])

df_amap=amap_2022.merge(city_popu,on='城市',how='inner')

df_amap['traffic_status_lastXd'] = df_amap.groupby('城市')['traffic_status'].transform(lambda x: x.rolling(1, 1).mean())

def catgorize2(g):
    level='5 社会面流动性无影响'
    g=float(g)
    if g <=-0.25:
        level= '0 社会面基本停滞'
    elif -0.25<g<=-0.15:
        level ='1 社会面流动性显著降低'
    elif -0.15<g<=-0.05:
        level ='4 社会面流动性轻微降低'
    elif g>-0.05:
        level ='5 社会面流动性无影响'
    return level

df_amap['traffic_change_level']=df_amap['traffic_status_lastXd'].apply(catgorize2)

df_amap['date']=pd.to_datetime(df_amap['date'])
df_amap['WeekDay']=df_amap['date'].dt.strftime('%A')
c1=df_amap['WeekDay'].isin(['Saturday','Sunday'])

pv_value=pd.pivot_table(df_amap[~c1], values='常住人口', index=['date'],columns=["traffic_change_level"], aggfunc=np.sum)
pv_value=pv_value.div(pv_value.sum(axis=1), axis=0)
pv_value['sum']=1
pv_value=pv_value[['sum','0 社会面基本停滞', '1 社会面流动性显著降低', '4 社会面流动性轻微降低', '5 社会面流动性无影响']]
pv_value=pv_value.fillna(0)
pv_value.tail(22).to_clipboard()
pv_value.tail(5)

traffic_change_level,sum,0 社会面基本停滞,1 社会面流动性显著降低,4 社会面流动性轻微降低,5 社会面流动性无影响
date,,,,,
2022-12-01,1,0.146193,0.127753,0.153762,0.572292
2022-12-02,1,0.152380,0.087864,0.131714,0.628042
2022-12-05,1,0.069899,0.109727,0.136540,0.683834
2022-12-06,1,0.069899,0.105375,0.212421,0.612305
2022-12-07,1,0.055317,0.126145,0.183751,0.634787


In [17]:
#社会面影响程度，典型城市
show=df_amap
show['date']=show['date'].astype('string') 
l=show[show['date']=='2022-12-01']
l=l.sort_values(by=['traffic_change_level','常住人口'],ascending=False)
table=l.groupby('traffic_change_level').head(5)[['traffic_change_level','date', '城市','traffic_status_last7d', '常住人口']].sort_values(by=['traffic_change_level','traffic_status_last7d','常住人口'],ascending=[True,True,False])
table.to_clipboard()
table.head(5)

,traffic_change_level,date,城市,traffic_status_last7d,常住人口
25418,0 社会面基本停滞,2022-12-01,重庆,-0.365539,2542
24062,0 社会面基本停滞,2022-12-01,北京,-0.358164,2189
18299,0 社会面基本停滞,2022-12-01,石家庄,-0.282321,576
6773,0 社会面基本停滞,2022-12-01,保定,-0.263993,331
20672,0 社会面基本停滞,2022-12-01,哈尔滨,-0.257154,707


In [18]:
# 航班数据
# flights=pd.read_excel(r'./DataSource/航班.xlsx').to_hdf('./flights.h5',key='raw')
flights=pd.read_hdf('./flights.h5',key='raw')
flights=pd.melt(flights,id_vars=['城市','机场'],var_name='date',value_name='flights')
df_covid['date'] = pd.to_datetime(df_covid['date']).dt.date
flights['date'] = pd.to_datetime(flights['date']).dt.date
# df_covid.dtypes
flights=flights.merge(df_covid,on=['城市','date'])
flights=flights.sort_values(by=['城市','date'])
flights['flights_last7d_mean']=flights.groupby('城市')['flights'].transform(lambda x: x.rolling(7, 7).mean())
flights['flights_WoW']=flights['flights_last7d_mean']/flights.groupby('城市')['flights_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

# covid rank vs. traffic change
flights['date']=flights['date'].astype('string') 
flights['covid_rank']=flights.groupby(by=['date'])['cases_last7d_sum'].rank(method="dense", ascending=True)
covidrank_fightschange=flights[flights['date']=='2022-12-01'][['城市','covid_rank','flights_WoW']] #,'case_growth_7','cases_last7d_sum','cases_14_7d_sum'
covidrank_fightschange.to_clipboard(index=False,header=None)


In [73]:
pd.pivot_table(hotel1,index=['cityName','常住人口'],columns='date',values=['bookedNights','bookedNights_change_WoW'], aggfunc=np.sum).to_clipboard()